In [1]:
import pandas as pd
import numpy as np

In [89]:
class NaiveBayesionClassifier:
    
    def __init__(self):
        pass
    
    def fit(self, Xtrain: pd.DataFrame, Ytrain: pd.Series):
        n_sample = len(Ytrain) # 样本数量
        clases = Ytrain.unique() # 所有类别
        n_clas = len(clases) # 类别数量
        clas_sample_num = Ytrain.value_counts() # 不同类别样本的数量
        clas_probs = {} # 数据集中各类样本出现的概率
        # 计算数据集中各类样本出现的概率
        for clas in clases:
            clas_probs[clas] = (clas_sample_num[clas] + 1) / (n_sample + n_clas)
        self.clas_sample_num = clas_sample_num
        self.n_clas = n_clas
        self.clases = clases
        self.attrs = Xtrain.columns.values # 所有属性
        self.attrval_num = Xtrain.apply(lambda s: len(s.unique())) # 各属性的取值数
        self.Xtrain = Xtrain
        self.Ytrain = Ytrain
        self.clas_probs = clas_probs
        
    def predict_1(self, x: pd.Series):
        prob_disbt = pd.Series(np.ones(self.n_clas), index=self.clases) # 测试样本的概率分布
        for clas in self.clases:
            prob = 1
            for attr in self.attrs:
                num = ((self.Xtrain[attr] == x[attr]) & (self.Ytrain == clas)).sum() # 类别clas中属性attr取值为x[attr]的数量
                prob *= (num + 1) / (self.clas_sample_num[clas] + self.attrval_num[attr]) # 属性条件概率
            prob_disbt[clas] = self.clas_probs[clas] * prob
        pred = prob_disbt.idxmax() # 概率最大的类别作为预测结果
        return pred
    
    def predict_n(self, Xtest: pd.DataFrame):
        Ypred = Xtest.apply(self.predict_1, axis=1)
        return Ypred

In [90]:
# 测试
df1 = pd.read_csv('西瓜2.0.csv').drop('编号', axis=1)
Xtrain = df1.drop('好瓜', axis=1)
Ytrain = df1['好瓜']
nbc = NaiveBayesionClassifier()
nbc.fit(Xtrain, Ytrain)

In [92]:
df2 = pd.read_csv('西瓜测试.csv').drop('编号', axis=1)
Xtest = df2.drop('好瓜', axis=1)
Ytest = df2['好瓜']
nbc.predict_n(Xtest)

0    是
1    否
2    否
3    否
dtype: object